In [1]:
import os
import glob
import itertools

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial.distance import cdist

import sys
sys.path.append('..')

import modules.images as im
import modules.general as gen
import modules.pose_estimation as pe

In [2]:
lower_part_types = ['HEAD', 'HIP', 'UPPER_LEG', 'KNEE', 'LOWER_LEG', 'FOOT']

load_dir = os.path.join('..', 'data', 'kinect', 'processed', 'hypothesis')

# All files with .pkl extension
file_paths = glob.glob(os.path.join(load_dir, '*.pkl'))
file_names = [os.path.basename(x) for x in file_paths]

trial_id = '2014-12-16_P005_Post_004'
load_path = os.path.join(load_dir, trial_id) + '.pkl'

df = pd.read_pickle(load_path)

In [3]:
# Select frames with data
lower_parts, part_labels = gen.strings_with_any_substrings(
    df.columns, lower_part_types)

df_lower = df[lower_parts].dropna(axis=0)

population_series = df_lower.apply(
    lambda row: pe.get_population(row, part_labels)[0], axis=1)

label_series = df_lower.apply(
    lambda row: pe.get_population(row, part_labels)[1], axis=1)

In [4]:
foot_label = len(lower_part_types) - 1

func = lambda x: x[0][label_series.loc[x.Frame] == foot_label]

# All foot positions
foot_series = population_series.reset_index().apply(func, axis=1)
foot_series.index = population_series.index

In [5]:
# Max foot-to-foot distance at each frame
max_foot_distances = foot_series.apply(lambda x: cdist(x, x).max())

In [6]:
%matplotlib auto
plt.plot(max_foot_distances)
plt.grid()

Using matplotlib backend: MacOSX


In [7]:
best_pos_dir = os.path.join('..', 'data', 'kinect', 'best pos')
best_pos_path = os.path.join(best_pos_dir, trial_id) + '.pkl'

df_head_feet = pd.read_pickle(best_pos_path)

foot_dist = (df_head_feet.L_FOOT - df_head_feet.R_FOOT).apply(np.linalg.norm)

In [8]:
plt.plot(max_foot_distances)
plt.plot(foot_dist)

In [9]:
dists = [57.8, 56.57, 58.77, 60.8, 61.5, 57.75, 57.93]